# Kaggle eCommerce Behavior Analysis
This Notebook briefly analyzes the [Kaggle eCommerce Behavior dataset](https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store)

In [1]:
import kaggle
import os
from pprint import pprint

kaggle.api.authenticate()

dataset_folder = "dataset"
dataset_author = "mkechinov"
dataset_name = "ecommerce-behavior-data-from-multi-category-store"
dataset_full_name = f"{dataset_author}/{dataset_name}"
os.makedirs(dataset_folder, exist_ok=True)

print("dataset_list_files_cli")
print(kaggle.api.dataset_list_files_cli(dataset_full_name))
print("\ndataset_list_files")
print(kaggle.api.dataset_list_files(dataset_full_name).files)
print("\ndatasets_list_files")
for file in kaggle.api.datasets_list_files(dataset_author, dataset_name)['datasetFiles']:
    pprint(file)

# kaggle.api.dataset_download_files(dataset_name, path=dataset_folder, unzip=True, quiet=False)

dataset_list_files_cli
name          size  creationDate         
------------  ----  -------------------  
2019-Nov.csv   8GB  2019-12-09 20:52:13  
2019-Oct.csv   5GB  2019-12-09 20:51:21  
None

dataset_list_files
[, ]

datasets_list_files
{'columns': [],
 'creationDate': '2019-12-09T20:52:13.431Z',
 'datasetRef': '',
 'datasetRefNullable': None,
 'description': '',
 'descriptionNullable': None,
 'fileType': '',
 'fileTypeNullable': None,
 'hasDatasetRef': False,
 'hasDescription': False,
 'hasFileType': False,
 'hasName': True,
 'hasOwnerRef': False,
 'hasUrl': False,
 'name': '2019-Nov.csv',
 'nameNullable': '2019-Nov.csv',
 'ownerRef': '',
 'ownerRefNullable': None,
 'ref': '',
 'totalBytes': 9006762395,
 'url': '',
 'urlNullable': None}
{'columns': [],
 'creationDate': '2019-12-09T20:51:21.345Z',
 'datasetRef': '',
 'datasetRefNullable': None,
 'description': '',
 'descriptionNullable': None,
 'fileType': '',
 'fileTypeNullable': None,
 'hasDatasetRef': False,
 'hasDescription': 

In [2]:
# Import SparkSession
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType

# Create SparkSession 
spark = SparkSession.builder \
      .master("local[*]") \
      .appName("e_commerce_behavior") \
      .getOrCreate() 
# spark = SparkSession.builder.appName("e_commerce_behavior").config("spark.memory.offHeap.enabled","true").config("spark.memory.offHeap.size","10g").getOrCreate()
# Create RDD from external Data source
df_sept = spark.read.csv(f"{dataset_folder}/2019-Oct.csv",header=True,escape="\"")
df_sept_size = df_sept.count()
print(df_sept_size)
df_sept.show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/13 08:55:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/13 08:55:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


42448764
+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|          event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|
+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|2019-10-01 00:00:...|      view|  44600062|2103807459595387724|                NULL|shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|
|2019-10-01 00:00:...|      view|   3900821|2053013552326770905|appliances.enviro...|    aqua|  33.20|554748717|9333dfbd-b87a-470...|
|2019-10-01 00:00:...|      view|  17200506|2053013559792632471|furniture.living_...|    NULL| 543.10|519107250|566511c2-e2e3-422...|
|2019-10-01 00:00:...|      view|   1307067|2053013558920217191|  computers.notebook|  lenovo| 251.74|550050854|7c90fc70-0e80-459...|
|2019-10-01 00:00:...|      view|   1004237|205301355

In [3]:
df_sept.printSchema()

root
 |-- event_time: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_session: string (nullable = true)



In [4]:
df_nov = spark.read.csv(f"{dataset_folder}/2019-Nov.csv",header=True,escape="\"")
df_nov_size = df_nov.count()
print(df_nov_size)
df_nov.show()

67501979
+--------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|          event_time|event_type|product_id|        category_id|       category_code|   brand| price|  user_id|        user_session|
+--------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|2019-11-01 00:00:...|      view|   1003461|2053013555631882655|electronics.smart...|  xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|
|2019-11-01 00:00:...|      view|   5000088|2053013566100866035|appliances.sewing...|  janome|293.65|530496790|8e5f4f83-366c-4f7...|
|2019-11-01 00:00:...|      view|  17302664|2053013553853497655|                NULL|   creed| 28.31|561587266|755422e7-9040-477...|
|2019-11-01 00:00:...|      view|   3601530|2053013563810775923|appliances.kitche...|      lg|712.87|518085591|3bfb58cd-7892-48c...|
|2019-11-01 00:00:...|      view|   1004775|2053013555631882

In [5]:
df_nov.printSchema()

root
 |-- event_time: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_session: string (nullable = true)



In [6]:
df_merged = df_sept.union(df_nov)
df_merged_size = df_merged.count()
print(df_merged_size)
assert df_merged_size == (df_sept_size + df_nov_size)
df_merged.printSchema()

24/08/13 08:56:09 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


109950743
root
 |-- event_time: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_session: string (nullable = true)



In [7]:
df_merged.show()

+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|          event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|
+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|2019-10-01 00:00:...|      view|  44600062|2103807459595387724|                NULL|shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|
|2019-10-01 00:00:...|      view|   3900821|2053013552326770905|appliances.enviro...|    aqua|  33.20|554748717|9333dfbd-b87a-470...|
|2019-10-01 00:00:...|      view|  17200506|2053013559792632471|furniture.living_...|    NULL| 543.10|519107250|566511c2-e2e3-422...|
|2019-10-01 00:00:...|      view|   1307067|2053013558920217191|  computers.notebook|  lenovo| 251.74|550050854|7c90fc70-0e80-459...|
|2019-10-01 00:00:...|      view|   1004237|205301355563188265

In [8]:
df_merged_1 = df_merged.withColumn("price", df_merged.price.cast("double"))\
    .withColumn("event_time", df_merged.event_time.cast(DateType()))
    # .withColumn("user_id", df_merged.user_id.cast("int"))\
    # .withColumn("category_id", df_merged.category_id.cast("int"))\
    # .withColumn("product_id", df_merged.product_id.cast("int"))\
df_merged_1.printSchema()

root
 |-- event_time: date (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_session: string (nullable = true)



In [9]:
df_merged_1.show(truncate=False)

+----------+----------+----------+-------------------+-----------------------------------+--------+-------+---------+------------------------------------+
|event_time|event_type|product_id|category_id        |category_code                      |brand   |price  |user_id  |user_session                        |
+----------+----------+----------+-------------------+-----------------------------------+--------+-------+---------+------------------------------------+
|2019-10-01|view      |44600062  |2103807459595387724|NULL                               |shiseido|35.79  |541312140|72d76fde-8bb3-4e00-8c23-a032dfed738c|
|2019-10-01|view      |3900821   |2053013552326770905|appliances.environment.water_heater|aqua    |33.2   |554748717|9333dfbd-b87a-4708-9857-6336556b0fcc|
|2019-10-01|view      |17200506  |2053013559792632471|furniture.living_room.sofa         |NULL    |543.1  |519107250|566511c2-e2e3-422b-b695-cf8e6e792ca8|
|2019-10-01|view      |1307067   |2053013558920217191|computers.notebo

In [10]:
df_merged_1.select("event_type").distinct().show()

+----------+
|event_type|
+----------+
|  purchase|
|      view|
|      cart|
+----------+



In [11]:
df_merged_1.select("category_id").distinct().show()

+-------------------+
|        category_id|
+-------------------+
|2085718636156158307|
|2053013556252639687|
|2134905045328592909|
|2137134549706998477|
|2152167773222993940|
|2053013561696846593|
|2053013562250494749|
|2053013555413778833|
|2053013552888807671|
|2075962341790319330|
|2079713983761220055|
|2070004998778912979|
|2053013552603594983|
|2100064858975240628|
|2141229835995840847|
|2053013566587404305|
|2053013564968403895|
|2053013558190408249|
|2053013553140465927|
|2053013555657048481|
+-------------------+
only showing top 20 rows



In [12]:
df_merged_1.groupBy("brand").count().sort("count", ascending=False).show()

+--------+--------+
|   brand|   count|
+--------+--------+
|    NULL|15331243|
| samsung|13172020|
|   apple|10381933|
|  xiaomi| 7721825|
|  huawei| 2521331|
| lucente| 1840936|
|      lg| 1659394|
|   bosch| 1532149|
|    oppo| 1294585|
|    sony| 1255101|
|    acer| 1127129|
|  lenovo| 1065493|
|cordiant| 1039948|
| respect| 1036498|
|   artel| 1002484|
|      hp|  820594|
| redmond|  741768|
| philips|  738854|
| indesit|  718964|
|dauscher|  706026|
+--------+--------+
only showing top 20 rows



In [13]:
df_merged_1.summary("count", "min", "25%", "75%", "max").show()

24/08/13 08:57:00 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+----------+-----------+--------------------+-------------------+--------+---------+------------+--------------------+
|summary|event_type| product_id|         category_id|      category_code|   brand|    price|     user_id|        user_session|
+-------+----------+-----------+--------------------+-------------------+--------+---------+------------+--------------------+
|  count| 109950743|  109950743|           109950743|           74536963|94619500|109950743|   109950743|           109950731|
|    min|      cart|  100000000| 2053013552226107603|    accessories.bag|  a-case|      0.0|   100963605|00000042-3e3f-42f...|
|    25%|      NULL|  1005256.0|2.053013555355058...|               NULL|     NaN|    67.96|5.16262445E8|                NULL|
|    75%|      NULL|1.7200513E7|2.053013563651392...|               NULL|Infinity|   360.11|5.56330356E8|                NULL|
|    max|      view|    9900463| 2187707861038006932|stationery.cartrige|   zyxel|  2574.07|    97129396|fffffd